In [ ]:
!pip install pdfplumber docx2txt transformers torch scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/363.4 MB ? eta -:--:--

In [2]:
import pdfplumber
import docx2txt

def extract_text_from_file(file_path):
    if file_path.endswith(".pdf"):
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload PDF or DOCX.")
    return text

In [3]:
import re

def split_text_into_sentences(text):
    sentences = re.split(r'\n+|(?<=[.!?])\s+', text.strip())
    return [s.strip() for s in sentences if len(s.strip()) > 10]

In [4]:
# from transformers import pipeline

# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# labels = ["Education", "Experience", "Projects", "Skills", "Certifications"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
# def classify_cv_text(sentences):
#     results = []
#     for sentence in sentences:
#         pred = classifier(sentence, labels)
#         label = pred["labels"][0]
#         score = pred["scores"][0]
#         results.append({"text": sentence, "category": label, "confidence": round(score, 3)})
#     return results

In [7]:
file_path = "/content/Adham_Ashraf_Eltholth.pdf"
text = extract_text_from_file(file_path)
sentences = split_text_into_sentences(text)

['Adham Ashraf Eltholth',
 '+20 15 50840224 | adham.eltholth@gmail.com | October Gardens, Giza',
 'linkedin.com/in/adham-ashraf | github.com/aadhamashraf',
 'A multi-passionate computer science and AI undergraduate with a strong foundation in software engineering, machine',
 'learning, and intelligent systems.',
 'I aim to bridge the gap between innovation and implementation through meaningful,',
 'scalable, and impactful tech solutions.',
 'Passionate about AI explainability, full-stack development, and solving real-world',
 'problems with creative and detail-oriented thinking.',
 'Actively seeking opportunities to contribute to cutting-edge projects,',
 'grow through mentorship, and build futuristic, human-centric technology platforms.',
 '• Zewail City of Science, Technology and Innovation Aug.',
 '2022 – Sept.',
 'Bachelor of Science in Computer Science and Artificial Intelligence',
 '• Data Science and AI Intern July 2025 - Present',
 'Orange Innovation Egypt',
 'Worked on the dev

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

df = pd.read_csv("/kaggle/input/technical-cv-augmented-ollama-csv/technical_cv_augmented_ollama.csv")
assert 'text' in df.columns and 'label' in df.columns, "Dataset must contain 'text' and 'label' columns."
df['label'] = df['label'].astype(str)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

models = {
    "Decision Tree": DecisionTreeClassifier(max_depth=50, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=200, random_state=42)
}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    print(f"\n{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Training Decision Tree...

Decision Tree Accuracy: 0.9667
                precision    recall  f1-score   support

Certifications       0.98      0.99      0.98       154
     Education       1.00      0.99      1.00       158
    Experience       1.00      0.92      0.96       156
      Projects       0.88      0.99      0.93       153
        Skills       0.98      0.95      0.96       159

      accuracy                           0.97       780
     macro avg       0.97      0.97      0.97       780
  weighted avg       0.97      0.97      0.97       780

Confusion Matrix:
 [[152   0   0   2   0]
 [  0 157   0   0   1]
 [  0   0 143  13   0]
 [  0   0   0 151   2]
 [  3   0   0   5 151]]

Training Random Forest...

Random Forest Accuracy: 0.9500
                precision    recall  f1-score   support

Certifications       1.00      1.00      1.00       154
     Education       1.00      0.99      1.00       158
    Experience       1.00      0.89      0.94       156
      Projects 

In [24]:
sample_text = "Developed a neural network for computer vision tasks"
sample_vector = vectorizer.transform([sample_text])
best_model = models["Gradient Boosting"]
print("\nPredicted label:", best_model.predict(sample_vector)[0])

hard_samples = [
    ("Developed a neural network for image recognition during my Master’s at MIT.", "Projects"),
    ("Worked as a Data Analyst intern while completing my bachelor’s in Computer Science.", "Experience"),
    ("Implemented a web app to visualize stock data using React and Python.", "Projects"),
    ("Earned AWS Certified Solutions Architect after developing multiple cloud-based apps.", "Certifications"),
    ("Trained in machine learning through Coursera and applied concepts in a deep learning project.", "Education"),
    ("Led a team of 4 engineers to design a mobile app used by 10K users.", "Experience"),
    ("Built and deployed a recommendation system during an internship at Google.", "Experience"),
    ("Completed a deep learning specialization from Andrew Ng on Coursera.", "Certifications"),
    ("Skilled in Python, TensorFlow, and REST API development.", "Skills"),
    ("Graduated with honors in Electrical Engineering, focusing on control systems.", "Education"),
    ("Designed and implemented a CI/CD pipeline for automated model deployment.", "Projects"),
    ("Volunteered as a robotics instructor, mentoring students in Arduino projects.", "Experience"),
    ("Achieved Microsoft Certified: Azure Fundamentals credential.", "Certifications"),
    ("Developed a mobile fitness tracker app with Flutter and Firebase.", "Projects"),
    ("Mentored junior developers on Python and Git best practices.", "Experience"),
    ("Completed Bachelor’s in Artificial Intelligence with a capstone on NLP models.", "Education"),
    ("Proficient in SQL, Docker, and data visualization tools like Tableau.", "Skills"),
    ("Built an automated resume parser using NLP libraries.", "Projects"),
    ("Obtained TensorFlow Developer Certificate and implemented CNNs.", "Certifications"),
    ("Implemented real-time anomaly detection system for IoT devices during coursework.", "Projects")
]

df_hard = pd.DataFrame(hard_samples, columns=["text", "expected_label"])
X_hard_tfidf = vectorizer.transform(df_hard["text"])
df_hard["predicted_label"] = best_model.predict(X_hard_tfidf)
accuracy = (df_hard["predicted_label"] == df_hard["expected_label"]).mean()
print(f"\nHard Test Accuracy: {accuracy:.2f}")
print(df_hard)
df_hard.to_csv("cv_hard_test_predictions.csv", index=False)
print("Saved hard test results to 'cv_hard_test_predictions.csv'")


Predicted label: Projects

Hard Test Accuracy: 0.50
                                                 text  expected_label  \
0   Developed a neural network for image recogniti...        Projects   
1   Worked as a Data Analyst intern while completi...      Experience   
2   Implemented a web app to visualize stock data ...        Projects   
3   Earned AWS Certified Solutions Architect after...  Certifications   
4   Trained in machine learning through Coursera a...       Education   
5   Led a team of 4 engineers to design a mobile a...      Experience   
6   Built and deployed a recommendation system dur...      Experience   
7   Completed a deep learning specialization from ...  Certifications   
8   Skilled in Python, TensorFlow, and REST API de...          Skills   
9   Graduated with honors in Electrical Engineerin...       Education   
10  Designed and implemented a CI/CD pipeline for ...        Projects   
11  Volunteered as a robotics instructor, mentorin...      Experience  

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

df = pd.read_csv("/kaggle/input/technical-cv-augmented-ollama-csv/technical_cv_augmented_ollama.csv")

assert 'text' in df.columns and 'label' in df.columns, "Dataset must contain 'text' and 'label' columns."

df['label'] = df['label'].astype(str)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1,2)  # Unigrams + bigrams for better feature richness
)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf = LogisticRegression(max_iter=200)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


sample_text = "Developed a neural network for computer vision tasks"
sample_vector = vectorizer.transform([sample_text])
print("\nPredicted label:", clf.predict(sample_vector)[0])


Accuracy: 1.0

Classification Report:
                 precision    recall  f1-score   support

Certifications       1.00      1.00      1.00       158
     Education       1.00      1.00      1.00       170
    Experience       1.00      1.00      1.00       166
      Projects       1.00      1.00      1.00       127
        Skills       1.00      1.00      1.00       159

      accuracy                           1.00       780
     macro avg       1.00      1.00      1.00       780
  weighted avg       1.00      1.00      1.00       780


Confusion Matrix:
 [[158   0   0   0   0]
 [  0 170   0   0   0]
 [  0   0 166   0   0]
 [  0   0   0 127   0]
 [  0   0   0   0 159]]

Predicted label: Projects


In [27]:
hard_samples = [
    ("Developed a neural network for image recognition during my Master’s at MIT.", "Projects"),
    ("Worked as a Data Analyst intern while completing my bachelor’s in Computer Science.", "Experience"),
    ("Implemented a web app to visualize stock data using React and Python.", "Projects"),
    ("Earned AWS Certified Solutions Architect after developing multiple cloud-based apps.", "Certifications"),
    ("Trained in machine learning through Coursera and applied concepts in a deep learning project.", "Education"),
    ("Led a team of 4 engineers to design a mobile app used by 10K users.", "Experience"),
    ("Built and deployed a recommendation system during an internship at Google.", "Experience"),
    ("Completed a deep learning specialization from Andrew Ng on Coursera.", "Certifications"),
    ("Skilled in Python, TensorFlow, and REST API development.", "Skills"),
    ("Graduated with honors in Electrical Engineering, focusing on control systems.", "Education"),
    ("Designed and implemented a CI/CD pipeline for automated model deployment.", "Projects"),
    ("Volunteered as a robotics instructor, mentoring students in Arduino projects.", "Experience"),
    ("Achieved Microsoft Certified: Azure Fundamentals credential.", "Certifications"),
    ("Developed a mobile fitness tracker app with Flutter and Firebase.", "Projects"),
    ("Mentored junior developers on Python and Git best practices.", "Experience"),
    ("Completed Bachelor’s in Artificial Intelligence with a capstone on NLP models.", "Education"),
    ("Proficient in SQL, Docker, and data visualization tools like Tableau.", "Skills"),
    ("Built an automated resume parser using NLP libraries.", "Projects"),
    ("Obtained TensorFlow Developer Certificate and implemented CNNs.", "Certifications"),
    ("Implemented real-time anomaly detection system for IoT devices during coursework.", "Projects")
]

df_hard = pd.DataFrame(hard_samples, columns=["text", "expected_label"])
X_hard_tfidf = vectorizer.transform(df_hard["text"])
df_hard["predicted_label"] = clf.predict(X_hard_tfidf)
accuracy = (df_hard["predicted_label"] == df_hard["expected_label"]).mean()
print(f"\nHard Test Accuracy: {accuracy:.2f}")
print(df_hard)
df_hard.to_csv("cv_hard_test_predictions.csv", index=False)
print("Saved hard test results to 'cv_hard_test_predictions.csv'")


Hard Test Accuracy: 0.75
                                                 text  expected_label  \
0   Developed a neural network for image recogniti...        Projects   
1   Worked as a Data Analyst intern while completi...      Experience   
2   Implemented a web app to visualize stock data ...        Projects   
3   Earned AWS Certified Solutions Architect after...  Certifications   
4   Trained in machine learning through Coursera a...       Education   
5   Led a team of 4 engineers to design a mobile a...      Experience   
6   Built and deployed a recommendation system dur...      Experience   
7   Completed a deep learning specialization from ...  Certifications   
8   Skilled in Python, TensorFlow, and REST API de...          Skills   
9   Graduated with honors in Electrical Engineerin...       Education   
10  Designed and implemented a CI/CD pipeline for ...        Projects   
11  Volunteered as a robotics instructor, mentorin...      Experience   
12  Achieved Microsoft Ce

In [33]:
!pip install -U sentence-transformers==2.6.1 transformers==4.43.3 huggingface-hub==0.24.6


In [34]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

embedder = SentenceTransformer("all-MiniLM-L6-v2")

X_train_emb = embedder.encode(X_train.tolist(), show_progress_bar=True)
X_test_emb = embedder.encode(X_test.tolist(), show_progress_bar=True)

clf = LogisticRegression(max_iter=300)
clf.fit(X_train_emb, y_train)

y_pred = clf.predict(X_test_emb)
print("Embedding-based Accuracy:", accuracy_score(y_test, y_pred))

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-690c9f5c-24cb699b167b41cd10a55b10;e04b5906-5426-4c07-bf4b-5eabdf60add0)

Entry Not Found for url: https://huggingface.co/api/models/sentence-transformers/all-MiniLM-L6-v2/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"